<a href="https://colab.research.google.com/github/Krishnaa548/Mlops/blob/main/cleintserver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import socket
import threading
import random
import time
import ipywidgets as widgets
from IPython.display import display


class SimpleServer:
    def __init__(self):
        self.host = '127.0.0.1'
        self.port = random.randint(10000, 65000)  # Random port to avoid conflicts
        self.socket = None
        self.running = False
        self.clients = []

    def start(self):
        try:
            self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            self.socket.bind((self.host, self.port))
            self.socket.listen(5)
            self.running = True

            # Start accepting clients in background
            thread = threading.Thread(target=self.accept_clients)
            thread.daemon = True
            thread.start()

            return f"Server started on port {self.port}"
        except Exception as e:
            return f"Server error: {str(e)}"

    def accept_clients(self):
        self.socket.settimeout(1)  # Non-blocking with timeout
        while self.running:
            try:
                client, address = self.socket.accept()
                thread = threading.Thread(target=self.handle_client, args=(client,))
                thread.daemon = True
                thread.start()
                self.clients.append(client)
            except socket.timeout:
                pass
            except:
                if self.running:
                    print("Error accepting client")
                break

    def handle_client(self, client):
        while self.running:
            try:
                data = client.recv(1024).decode('utf-8')
                if not data:
                    break

                # Echo the message back
                response = f"Server echo: {data}"
                client.send(response.encode('utf-8'))
            except:
                break

        if client in self.clients:
            self.clients.remove(client)
        client.close()

    def stop(self):
        self.running = False

        # Close all client connections
        for client in self.clients:
            try:
                client.close()
            except:
                pass
        self.clients = []

        # Close server socket
        if self.socket:
            try:
                self.socket.close()
            except:
                pass
            self.socket = None

        return "Server stopped"

# Simple client class
class SimpleClient:
    def __init__(self):
        self.host = '127.0.0.1'
        self.port = None
        self.socket = None
        self.connected = False

    def connect(self, port):
        try:
            self.port = int(port)
            self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.socket.connect((self.host, self.port))
            self.connected = True
            return f"Connected to server on port {self.port}"
        except Exception as e:
            return f"Connection error: {str(e)}"

    def send(self, message):
        if not self.connected:
            return "Not connected to server"

        try:
            self.socket.send(message.encode('utf-8'))
            response = self.socket.recv(1024).decode('utf-8')
            return response
        except Exception as e:
            self.disconnect()
            return f"Send error: {str(e)}"

    def disconnect(self):
        self.connected = False
        if self.socket:
            try:
                self.socket.close()
            except:
                pass
            self.socket = None
        return "Disconnected from server"

# Create interactive UI
def create_simple_demo():
    # Create server and client objects
    server = SimpleServer()
    client = SimpleClient()

    # Create output area
    output = widgets.Output(layout={'border': '1px solid black', 'padding': '10px', 'height': '200px', 'overflow_y': 'auto'})

    # Server controls
    start_btn = widgets.Button(description="Start Server", button_style='success')
    stop_btn = widgets.Button(description="Stop Server", button_style='danger')
    server_status = widgets.HTML(value="<b>Server:</b> Not running")

    # Client controls
    connect_btn = widgets.Button(description="Connect Client", button_style='primary')
    disconnect_btn = widgets.Button(description="Disconnect Client", button_style='warning')
    client_status = widgets.HTML(value="<b>Client:</b> Not connected")

    # Message controls
    message_input = widgets.Text(placeholder='Type a message', layout={'width': '70%'})
    send_btn = widgets.Button(description="Send", button_style='info')

    # Button callbacks
    def on_start_clicked(b):
        with output:
            result = server.start()
            print(result)
            server_status.value = f"<b>Server:</b> Running on port {server.port}"

    def on_stop_clicked(b):
        with output:
            result = server.stop()
            print(result)
            server_status.value = "<b>Server:</b> Not running"

    def on_connect_clicked(b):
        with output:
            if not server.running:
                print("Server is not running. Start the server first.")
                return

            result = client.connect(server.port)
            print(result)
            client_status.value = f"<b>Client:</b> Connected to port {server.port}"

    def on_disconnect_clicked(b):
        with output:
            result = client.disconnect()
            print(result)
            client_status.value = "<b>Client:</b> Not connected"

    def on_send_clicked(b):
        with output:
            if not message_input.value:
                print("Please enter a message")
                return

            result = client.send(message_input.value)
            print(f"Sent: {message_input.value}")
            print(f"Received: {result}")
            message_input.value = ""

    # Connect callbacks
    start_btn.on_click(on_start_clicked)
    stop_btn.on_click(on_stop_clicked)
    connect_btn.on_click(on_connect_clicked)
    disconnect_btn.on_click(on_disconnect_clicked)
    send_btn.on_click(on_send_clicked)

    # Layout
    server_box = widgets.HBox([start_btn, stop_btn, server_status])
    client_box = widgets.HBox([connect_btn, disconnect_btn, client_status])
    message_box = widgets.HBox([message_input, send_btn])

    # Display the UI
    display(widgets.HTML("<h2>Simple Client-Server Demo</h2>"))
    display(server_box)
    display(client_box)
    display(message_box)
    display(output)

    def on_enter(sender):
        on_send_clicked(None)

    message_input.on_submit(on_enter)

# Run the demo
create_simple_demo()


HTML(value='<h2>Simple Client-Server Demo</h2>')

Output(layout=Layout(border='1px solid black', height='200px', overflow_y='auto', padding='10px'))